In [ ]:
import json
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Paths to your dataset
image_dir = "../../advance/images"
annotations_path = "../../advance/vlm.jsonl"

# Load annotations
annotations = []
with open(annotations_path, 'r') as f:
    for line in f:
        annotations.append(json.loads(line))

print(f"Loaded {len(annotations)} annotations.")

# Load images and corresponding annotations
def load_data(annotations, image_dir):
    images = []
    captions = []
    bboxes = []

    for annotation in annotations:
        image_path = os.path.join(image_dir, annotation['image'])
        image = load_img(image_path, target_size=(224, 224))
        image = img_to_array(image) / 255.0

        for ann in annotation['annotations']:
            captions.append(ann['caption'])
            bboxes.append(ann['bbox'])
        
        images.append(image)
    
    print(f"Loaded {len(images)} images.")
    return np.array(images), captions, bboxes

images, captions, bboxes = load_data(annotations, image_dir)


2024-05-18 07:57:50.499689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded 5107 annotations.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize captions
tokenizer = Tokenizer(num_words=10000, oov_token="<UNK>")
tokenizer.fit_on_texts(captions)
sequences = tokenizer.texts_to_sequences(captions)
padded_sequences = pad_sequences(sequences, padding='post')

print(f"Tokenized {len(captions)} captions into sequences.")

# Normalize bounding boxes
def normalize_bbox(bbox, image_shape):
    height, width, _ = image_shape
    x, y, w, h = bbox
    return [x / width, y / height, w / width, h / height]

normalized_bboxes = [normalize_bbox(bbox, images[0].shape) for bbox in bboxes]
print(f"Normalized {len(normalized_bboxes)} bounding boxes.")


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Concatenate, Flatten

# Image feature extractor
image_input = Input(shape=(224, 224, 3))
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=image_input)
x = Flatten()(base_model.output)
image_features = Dense(256, activation='relu')(x)

# Caption processor
caption_input = Input(shape=(None,))
embedding = Embedding(input_dim=10000, output_dim=256, mask_zero=True)(caption_input)
lstm_out = LSTM(256)(embedding)

# Bounding box predictor
combined = Concatenate()([image_features, lstm_out])
bbox_output = Dense(4, activation='sigmoid')(combined)

# Define the model
model = Model(inputs=[image_input, caption_input], outputs=bbox_output)
model.compile(optimizer='adam', loss='mse')

print("Model Summary:")
model.summary()


In [ ]:
# Prepare the inputs
def prepare_inputs(images, sequences, normalized_bboxes, max_seq_length):
    images_input = np.array(images)
    sequences_input = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    bboxes_output = np.array(normalized_bboxes)
    
    return images_input, sequences_input, bboxes_output

max_seq_length = max(len(seq) for seq in sequences)
images_input, sequences_input, bboxes_output = prepare_inputs(images, sequences, normalized_bboxes, max_seq_length)

print(f"Prepared inputs: images_input shape = {images_input.shape}, sequences_input shape = {sequences_input.shape}, bboxes_output shape = {bboxes_output.shape}")

# Train the model
model.fit([images_input, sequences_input], bboxes_output, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Further fine-tuning can be done by adjusting learning rates, using callbacks, etc.
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.1, patience=2)
]

history = model.fit([images_input, sequences_input], bboxes_output, epochs=50, batch_size=32, validation_split=0.2, callbacks=callbacks)

print(f"Training completed. History: {history.history}")

# Evaluate the model
val_loss = model.evaluate([images_input, sequences_input], bboxes_output)
print(f"Validation Loss: {val_loss}")


In [ ]:
import json
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths to your dataset
image_dir = "../../advance/images"
annotations_path = "../../advance/vlm.jsonl"

# Load annotations
annotations = []
with open(annotations_path, 'r') as f:
    for line in f:
        annotations.append(json.loads(line))

print(f"Loaded {len(annotations)} annotations.")

# Select one random annotation
random_annotation = random.choice(annotations)
image_path = os.path.join(image_dir, random_annotation['image'])
image = load_img(image_path, target_size=(224, 224))
image = img_to_array(image) / 255.0

print(f"Loaded random image: {random_annotation['image']}")

# Get captions and bounding boxes for the selected image
captions = [ann['caption'] for ann in random_annotation['annotations']]
bboxes = [ann['bbox'] for ann in random_annotation['annotations']]
print(f"Captions: {captions}")
print(f"Bounding Boxes: {bboxes}")


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize captions
tokenizer = Tokenizer(num_words=10000, oov_token="<UNK>")
tokenizer.fit_on_texts(captions)
sequences = tokenizer.texts_to_sequences(captions)
padded_sequences = pad_sequences(sequences, padding='post')

print(f"Tokenized captions: {sequences}")

# Normalize bounding boxes
def normalize_bbox(bbox, image_shape):
    height, width, _ = image_shape
    x, y, w, h = bbox
    return [x / width, y / height, w / width, h / height]

normalized_bboxes = [normalize_bbox(bbox, image.shape) for bbox in bboxes]
print(f"Normalized bounding boxes: {normalized_bboxes}")


In [ ]:
# Prepare the inputs
images_input = np.expand_dims(image, axis=0)
sequences_input = pad_sequences(sequences, maxlen=pad_sequences(sequences).shape[1], padding='post')
sequences_input = np.expand_dims(sequences_input[0], axis=0)  # Make it batch-compatible

print(f"Prepared inputs: images_input shape = {images_input.shape}, sequences_input shape = {sequences_input.shape}")

# Load the model (assumes model definition and compilation from earlier)
# model = ...  # Define and compile the model as shown earlier

# Make predictions
predicted_bboxes = model.predict([images_input, sequences_input])
print(f"Predicted bounding boxes: {predicted_bboxes}")
